In [ ]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Function to read and preprocess images and labels
def preprocess_data(data_dir, labels_dir):
    images = []
    labels = []
    for filename in os.listdir(labels_dir):
        if filename.endswith('.txt'):
            label_file = os.path.join(labels_dir, filename)
            with open(label_file, 'r') as file:
                for line in file:
                    line = line.strip().split()
                    image_path = os.path.join(data_dir, line[0])
                    image = cv2.imread(image_path)
                    image = cv2.resize(image, (224, 224))  # Resize images to 224x224
                    label = [float(x) for x in line[1:]]   # Convert label coordinates to float
                    labels.append(label)
                    images.append(image)
    return np.array(images), np.array(labels)


# Load data directories
train_images_dir = r'C:\Users\iamak\OneDrive\Documents\GCET\IPBL\train\images'
train_labels_dir = r'C:\Users\iamak\OneDrive\Documents\GCET\IPBL\train\labels'
test_images_dir = r'C:\Users\iamak\OneDrive\Documents\GCET\IPBL\test\images'
test_labels_dir = r'C:\Users\iamak\OneDrive\Documents\GCET\IPBL\test\labels'
val_images_dir = r'C:\Users\iamak\OneDrive\Documents\GCET\IPBL\valid\images'
val_labels_dir =  r'C:\Users\iamak\OneDrive\Documents\GCET\IPBL\valid\labels'

# Preprocess data
train_images, train_labels = preprocess_data(train_images_dir, train_labels_dir)
test_images, test_labels = preprocess_data(test_images_dir, test_labels_dir)
val_images, val_labels = preprocess_data(val_images_dir, val_labels_dir)

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=45,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    brightness_range=[0.5, 1.5],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Define models
def build_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return model

def build_resnet_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return model

def build_vgg_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return model

# Compile models
cnn_model = build_cnn_model()
resnet_model = build_resnet_model()
vgg_model = build_vgg_model()

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
resnet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
vgg_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train models
batch_size = 32
epochs = 50

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

cnn_history = cnn_model.fit(train_datagen.flow(train_images, train_labels, batch_size=batch_size),
                            steps_per_epoch=len(train_images) / batch_size,
                            epochs=epochs,
                            validation_data=val_datagen.flow(val_images, val_labels, batch_size=batch_size),
                            callbacks=[early_stopping])

resnet_history = resnet_model.fit(train_datagen.flow(train_images, train_labels, batch_size=batch_size),
                                  steps_per_epoch=len(train_images) / batch_size,
                                  epochs=epochs,
                                  validation_data=val_datagen.flow(val_images, val_labels, batch_size=batch_size),
                                  callbacks=[early_stopping])

vgg_history = vgg_model.fit(train_datagen.flow(train_images, train_labels, batch_size=batch_size),
                            steps_per_epoch=len(train_images) / batch_size,
                            epochs=epochs,
                            validation_data=val_datagen.flow(val_images, val_labels, batch_size=batch_size),
                            callbacks=[early_stopping])

# Evaluate models
cnn_loss, cnn_accuracy = cnn_model.evaluate(test_images, test_labels)
resnet_loss, resnet_accuracy = resnet_model.evaluate(test_images, test_labels)
vgg_loss, vgg_accuracy = vgg_model.evaluate(test_images, test_labels)

print("CNN Test Accuracy:", cnn_accuracy)
print("ResNet Test Accuracy:", resnet_accuracy)
print("VGG Test Accuracy:", vgg_accuracy)
